In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
#loading adult dataset
dataset = pd.read_csv('C:/Users/emnsrjm/Desktop/Courses/Machine Learning/Project/Machine-Learning-Project/classification-models/Adult/data/adult.data',header=None)
X = dataset.iloc[:, 1:14].values
Y = dataset.iloc[:, 14].values
State-gov, 77516, Bachelors, 13, Never-married, Adm-clerical, Not-in-family, White, Male, 2174, 0, 40, United-States
datatype = np.array(['D','C','D','C','D','D','D','D','D','C','C','C','D'])

In [51]:
# Preprocessing the discrete data
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for i in range(0,13):
    x_temp = X[:,i]
    if datatype[i]=='D':
        x_temp  = labelencoder.fit_transform(x_temp)
    if i==0:
        X_encoded = x_temp
    else:
        X_encoded = np.column_stack((x_new,x_temp))
Y_encoded = labelencoder.fit_transform(x_temp)

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y_encoded, test_size = 0.33, random_state = 0)

In [54]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

C:\Users\emnsrjm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\emnsrjm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\emnsrjm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\emnsrjm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [55]:
#Using Logistic Regression Algorithm to the Training Set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
print('Y test'+str(Y_test.shape))
print('Y pred'+str(Y_pred.shape))
accuracy = sklearn.metrics.accuracy_score(Y_test, Y_pred)
print('Logistic Regression :'+str(accuracy))

C:\Users\emnsrjm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\emnsrjm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Y test(10746,)
Y pred(10746,)
Logistic Regression :0.940070723990322
